In [1]:
AWS_ACCESS_KEY = ""
AWS_SECRET_KEY = ""
sc._jsc.hadoopConfiguration().set(
"fs.s3n.awsAccessKeyId", AWS_ACCESS_KEY)
sc._jsc.hadoopConfiguration().set(
"fs.s3n.awsSecretAccessKey", AWS_SECRET_KEY)

In [2]:
import time
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, \
                                      NaiveBayes, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import regexp_replace
import sys
import logging
import mlflow
import mlflow.mleap
import pandas as pd
import numpy as np

In [3]:
#spark = SparkSession.builder.appName("Network Attacks Classifier").master("local").getOrCreate()
#spark.sparkContext.setLogLevel("ERROR")
start = time.time()
dataset=spark.read.csv("s3://sparkml/networktrafficdata.csv",header=True, inferSchema = True)
dataset=dataset.toDF(*dataset.columns)

In [4]:
dataset = dataset.withColumn("label", regexp_replace("label", "\.", ""))
print("Dataset sizes: {row} samples, {cols} features".format(row=dataset.count(), cols=len(dataset.columns)))

<>:1: DeprecationWarning: invalid escape sequence \.
<>:1: DeprecationWarning: invalid escape sequence \.
<command-692560893596990>:1: DeprecationWarning: invalid escape sequence \.
 dataset = dataset.withColumn("label", regexp_replace("label", "\.", ""))
Dataset sizes: 4898431 samples, 42 features

In [5]:
dataset=dataset.withColumnRenamed(" flag","flag")

In [6]:
#Data Transformation Pipelining
categorical_features = ["protocol_type", "service", "flag"]
indexers = [StringIndexer(inputCol=column, outputCol=column + "_num") for column in categorical_features]
indexers.append(StringIndexer(inputCol="label", outputCol="label_num"))
pipeline = Pipeline(stages=indexers)
dataset = pipeline.fit(dataset).transform(dataset)

exclude_list = categorical_features + ["label", "label_num"]
numerical_cols = [col for col in dataset.columns if col not in exclude_list]


In [7]:
#Feature Engineering
#df_assembler = VectorAssembler(inputCols=numerical_cols, outputCol="features")
#dataset = df_assembler.transform(dataset)
# dataset.printSchema()

#dataset = dataset.select(["features","label_num"])
# dataset.printSchema()

#train_set, test_set = dataset.randomSplit([0.75, 0.25], seed=2019)
#print("Training set Count: " + str(train_set.count()))
#print("Test set Count: " + str(test_set.count()))

In [8]:
#Model Selection
# Logistic Regression model
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0.8, featuresCol="features",
                        labelCol="label_num", family="multinomial")

# Decision Tree model
dt = DecisionTreeClassifier(labelCol="label_num", featuresCol="features",  maxBins=70)

# Random Forest model
rf = RandomForestClassifier(labelCol="label_num", featuresCol="features", numTrees=20, maxBins=70)

classifiers = {"Logistic Regression": lr, "Decision Tree": dt,
               "Random Forest": rf}

metrics = ["accuracy", "weightedPrecision", "weightedRecall", "f1"]

In [9]:
# print("\nModels Evaluation:")
# print("{:-<30}".format(""))
# for c in classifiers:
# 	print(c)
# 	# fit the model
# 	model = classifiers[c].fit(train_set)
	
# 	# make predictions
# 	predictions = model.transform(test_set)
# 	predictions.cache()
	
# 	# evaluate performance
# 	evaluator = MulticlassClassificationEvaluator(labelCol="label_num", predictionCol="prediction")
	
# 	for m in metrics:
# 		evaluator.setMetricName(m)
# 		metric = evaluator.evaluate(predictions)
# 		print("{name} = {value:.2f}".format(name=m, value=metric))
	
# 	print("{:-<30}".format(""))

In [10]:
#Running the best model using mlflow
traindf, testdf = dataset.randomSplit([0.80, 0.20], seed=2020)
def trainmodel():
  df_assembler = VectorAssembler(inputCols=numerical_cols, outputCol="features")
  rfnew=RandomForestClassifier(labelCol="label_num", featuresCol="features", numTrees=30, maxDepth=10, maxBins=70)
  pl=Pipeline(stages=[df_assembler,rfnew])
  model=pl.fit(traindf)
  prediction=model.transform(testdf)
  evaluator = MulticlassClassificationEvaluator(labelCol="label_num", predictionCol="prediction")
  
  for m in metrics:
    evaluator.setMetricName(m)
    metric = evaluator.evaluate(prediction)
    mlflow.log_metric(m,metric)
    print("{name} = {value:.2f}".format(name=m, value=metric))
  
  
  mlflow.log_param("featuresCol", "features")
  mlflow.log_param("labelCol", "label_num")
  mlflow.log_param("numTrees", 30)
  mlflow.log_param("maxDepth", 10)
  mlflow.log_param("maxBins", 70)
  mlflow.mleap.log_model(spark_model=model, sample_input=testdf, artifact_path="spark_network_traffic_model")
  mlflow.end_run()
  

In [11]:
with mlflow.start_run() as run:
  trainmodel()

accuracy = 1.00
weightedPrecision = 1.00
weightedRecall = 1.00
f1 = 1.00

In [12]:
#import mlflow.sagemaker as mfs
#mlflow.get_artifact_uri('spark_network_traffic_model')
#mlflow.tracking.get_tracking_uri() 
#mfs.DEFAULT_IMAGE_NAME

In [13]:
mlflow.get_run('585995d94fbc469c820eda98ce80aa79')

Out[80]: <Run: data=<RunData: metrics={'accuracy': 0.9996098083157815,
 'f1': 0.9995639243773748,
 'weightedPrecision': 0.9996070252066136,
 'weightedRecall': 0.9996098083157813}, params={'featuresCol': 'features',
 'labelCol': 'label_num',
 'maxBins': '70',
 'maxDepth': '10',
 'numTrees': '30'}, tags={'mlflow.databricks.cluster.id': '0906-165337-timer312',
 'mlflow.databricks.cluster.info': '{"cluster_id":"0906-165337-timer312","creator_user_id":7151952614704296,"driver":{"public_dns":"ec2-18-237-58-126.us-west-2.compute.amazonaws.com","ui_uri":"http://10.172.247.22:43323","memory_mb":10347,"node_id":"0ff8ec8038644bf08cf885cfaece9202","node_aws_attributes":{"is_spot":false},"instance_id":"i-0e13ed76cf07a8068","start_timestamp":1599423659924,"host_private_ip":"10.172.245.188","private_ip":"10.172.247.22"},"driver_uri":"https://10.172.247.22:6060","driver_ui_uri":"http://10.172.247.22:43323","spark_context_id":8370450891800309979,"driver_healthy":true,"jdbc_port":10000,"cluster_name":"Ramnath","spark_version":"7.2.x-scala2.12","spark_conf":{"spark.databricks.delta.preview.enabled":"true"},"cluster_creator":"WEBAPP","aws_attributes":{"zone_id":"us-west-2c","first_on_demand":0,"availability":"ON_DEMAND","spot_bid_price_percent":100,"ebs_volume_count":0},"node_type_id":"dev-tier-node","driver_node_type_id":"dev-tier-node","spark_env_vars":{"PYSPARK_PYTHON":"/databricks/python3/bin/python3"},"autotermination_minutes":120,"enable_elastic_disk":false,"disk_spec":{"disk_count":0},"cluster_source":"UI","enable_jdbc_auto_start":true,"enable_jobs_autostart":true,"enable_local_disk_encryption":false,"package_label":"release__7.2.x-snapshot-scala2.12__databricks-universe__head__f6cc114__c3546c1__jenkins__89ae7c8__format-2","state":"RUNNING","state_message":"","start_time":1599411217559,"terminated_time":0,"last_state_loss_time":1599423716668,"last_command_completion_time":1599423680846,"last_activity_time":1599423680846,"creator_org_id":7474965780391700,"num_workers":0,"cluster_memory_mb":15616,"cluster_cores":2.0,"creator_user_name":"rramac9@uic.edu","acl_path":"/clusters/0906-165337-timer312","init_scripts_safe_mode":false}',
 'mlflow.databricks.notebookID': '2921259872676851',
 'mlflow.databricks.notebookPath': '/Users/rramac9@uic.edu/spark_network_traffic_classifier',
 'mlflow.databricks.notebookRevisionID': '1599450192994',
 'mlflow.databricks.webappURL': 'https://community.cloud.databricks.com',
 'mlflow.log-model.history': '[{"run_id":"585995d94fbc469c820eda98ce80aa79","artifact_path":"spark_network_traffic_model","utc_time_created":"2020-09-07 '
 '03:42:27.868063","flavors":{"mleap":{"mleap_version":"0.16.1","model_data":"mleap/model"}}}]',
 'mlflow.source.name': '/Users/rramac9@uic.edu/spark_network_traffic_classifier',
 'mlflow.source.type': 'NOTEBOOK',
 'mlflow.user': 'rramac9@uic.edu'}>, info=<RunInfo: artifact_uri='dbfs:/databricks/mlflow/2921259872676851/585995d94fbc469c820eda98ce80aa79/artifacts', end_time=1599450192888, experiment_id='2921259872676851', lifecycle_stage='active', run_id='585995d94fbc469c820eda98ce80aa79', run_uuid='585995d94fbc469c820eda98ce80aa79', start_time=1599449089860, status='FINISHED', user_id=''>>